In [51]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
import numpy as np
import pandas as pd

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [52]:
data_path = 'data/'

### load data

In [53]:
train = pd.read_csv(data_path + 'train.csv')
val = pd.read_csv(data_path + 'val.csv')
test = pd.read_csv(data_path + 'test.csv')

### Dataset

In [54]:
from torch.utils.data import Dataset, DataLoader

class PoliteDataset(Dataset):
    def __init__(self, X, y):
        self.text = X
        self.targets = y

    def __len__(self):
        return len(self.text)

    def __getitem__(self, idx):
        text = str(self.text[idx])
        target = int(self.targets[idx])

        return text, target

In [55]:
model_name = "airesearch/wangchanberta-base-att-spm-uncased"
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2)
tokenizer = AutoTokenizer.from_pretrained(model_name)

model.to(device)

Some weights of the model checkpoint at airesearch/wangchanberta-base-att-spm-uncased were not used when initializing CamembertForSequenceClassification: ['lm_head.dense.weight', 'lm_head.layer_norm.bias', 'lm_head.layer_norm.weight', 'roberta.pooler.dense.weight', 'lm_head.decoder.weight', 'roberta.pooler.dense.bias', 'lm_head.dense.bias', 'lm_head.decoder.bias', 'lm_head.bias']
- This IS expected if you are initializing CamembertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing CamembertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of CamembertForSequenceClassification were not initialized from the model checkpoint at airesearch/wa

CamembertForSequenceClassification(
  (roberta): CamembertModel(
    (embeddings): CamembertEmbeddings(
      (word_embeddings): Embedding(25005, 768, padding_idx=1)
      (position_embeddings): Embedding(512, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): CamembertEncoder(
      (layer): ModuleList(
        (0-11): 12 x CamembertLayer(
          (attention): CamembertAttention(
            (self): CamembertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): CamembertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=Tru

### second dataloaders

In [56]:
polite_train_dataset = PoliteDataset(train['text'].tolist(), train['label'].tolist())
train_loader = DataLoader(polite_train_dataset, batch_size=8, shuffle=True)
polite_val_dataset = PoliteDataset(val['text'].tolist(), val['label'].tolist())
val_loader = DataLoader(polite_val_dataset, batch_size=8, shuffle=True)
polite_test_dataset = PoliteDataset(test['text'].tolist(), test['label'].tolist())
test_loader = DataLoader(polite_test_dataset, batch_size=8)

### train

In [57]:
from transformers import AdamW

optimizer = AdamW(model.parameters(), lr=1e-5)

e:\Programs\miniconda3\envs\torch\lib\site-packages\transformers\optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [58]:
from tqdm import tqdm

In [ ]:
epochs = 10

for epoch in range(epochs):
    train_acc = 0
    print(f'Epoch {epoch}')
    for batch in tqdm(train_loader):
        text, target = batch
        encoding = tokenizer(list(text), return_tensors='pt', padding=True, truncation=True, max_length=256)
        encoding = encoding.to(device)
        target = target.to(device)
        output = model(**encoding, labels=target)
        loss = output.loss
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
        train_acc += (output.logits.argmax(dim=1) == target).sum()
    model.eval()
    test_acc = 0
    for batch in tqdm(test_loader):
        text, target = batch
        encoding = tokenizer(list(text), return_tensors='pt', padding=True, truncation=True, max_length=256)
        encoding = encoding.to(device)
        target = target.to(device)
        output = model(**encoding, labels=target)
        loss = output.loss
        test_acc += (output.logits.argmax(dim=1) == target).sum()
    model.save_pretrained(f'./checkpoints/classifier/new_{epoch}')
    print(f'Epoch {epoch} loss: {loss.item()} train accuracy: {train_acc/len(polite_train_dataset)} test accuracy: {test_acc/len(polite_test_dataset)}')

In [64]:
from sklearn.metrics import classification_report

def evaluate(model, test_loader):
    y_pred = []
    y_test = []

    model.eval()
    with torch.no_grad():
        for batch in tqdm(test_loader):
            text, target = batch
            encoding = tokenizer(list(text), return_tensors='pt', padding=True, truncation=True, max_length=256)
            output = model(**encoding, labels=target)

            pred = output.logits.argmax(dim=1)
            y_pred += pred.tolist()
            y_test += target.tolist()
    
    report = classification_report(y_pred, y_test)
    return report

In [60]:
model = AutoModelForSequenceClassification.from_pretrained('./checkpoints/classifier/new_8')

In [65]:
model.cpu()
report = evaluate(model, test_loader)
print(report)

100%|██████████| 397/397 [04:56<00:00,  1.34it/s]

              precision    recall  f1-score   support

           0       0.88      0.90      0.89      1547
           1       0.90      0.88      0.89      1623

    accuracy                           0.89      3170
   macro avg       0.89      0.89      0.89      3170
weighted avg       0.89      0.89      0.89      3170



In [66]:
model.eval()
inputs = tokenizer("มีเวลาก็หัดศึกษาเองบ้างสิ อีโง่", return_tensors="pt")
output = model(**inputs)
print(output.logits.argmax(dim=1))

tensor([0])


In [67]:
inputs = tokenizer("มีเวลาก็ลองศึกษาด้วยตัวเองดูนะคะ คุณผู้ไม่ฉลาด", return_tensors="pt")
output = model(**inputs)
print(output.logits.argmax(dim=1))

tensor([1])


# something

In [42]:
question, text = "Who was Jim Henson?", "Jim Henson was a nice puppet"

inputs = tokenizer(question, text, return_tensors="pt")
with torch.no_grad():
    outputs = model(**inputs)

answer_start_index = outputs.start_logits.argmax()
answer_end_index = outputs.end_logits.argmax()

predict_answer_tokens = inputs.input_ids[0, answer_start_index : answer_end_index + 1]
tokenizer.decode(predict_answer_tokens, skip_special_tokens=True)

# target is "nice puppet"
target_start_index = torch.tensor([14])
target_end_index = torch.tensor([15])

outputs = model(**inputs, start_positions=target_start_index, end_positions=target_end_index)
loss = outputs.loss
round(loss.item(), 2)

AttributeError: 'SequenceClassifierOutput' object has no attribute 'start_logits'

In [ ]:
text_token = tokenizer("สวัสดีค่ะ วันนี้เป็นวันจันทร์", return_tensors="pt")

In [ ]:
out = model2(**text_token)

In [ ]:
answer_start = torch.argmax(out.start_logits)
answer_end = torch.argmax(out.end_logits) + 1

In [ ]:
answer = tokenizer.convert_tokens_to_string(tokenizer.convert_ids_to_tokens(input_ids[answer_start:answer_end]))

In [ ]:
from transformers import AutoModelForCausalLM

In [ ]:
model3 = AutoModelForCausalLM.from_pretrained(model_name, is_decoder=True)

In [ ]:
inputs = tokenizer("วันนี้เป็นวันจันทร์ พรุ่งนี้เป็นวันอะไร", return_tensors="pt")

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM
# Load pre-trained RoBERTa model and tokenizer
model_name = "airesearch/wangchanberta-base-att-spm-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)

# Define input and output texts
input_text = "วันนี้วันจันทร์ พรุ่งนี้เป็นวันอะไร"
output_text = "วันพรุ่งนี้เป็นวันอังคาร"

# Tokenize inputs
input_ids = tokenizer.encode(input_text, return_tensors="pt")
output_ids = tokenizer.encode(output_text, return_tensors="pt")

# Generate output
generated_ids = model.generate(input_ids=input_ids, decoder_start_token_id=model.config.decoder.pad_token_id, max_length=128, num_beams=4, early_stopping=True)
generated_text = tokenizer.decode(generated_ids[0], skip_special_tokens=True)

print("Input text:", input_text)
print("Expected output text:", output_text)
print("Generated output text:", generated_text)
